## Live Training Environment

In [7]:
import pandas as pd
import numpy as np
import random
import time
from collections import deque
import yfinance as yf

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [3]:
ticker = yf.Ticker('GOOG').info['currentPrice']
ticker

107.82

In [5]:
class Agent:
        def __init__(self):
            self.state_size = 5 
            self.action_size = 3 # sit, buy, sell
            self.memory = deque(maxlen=1000)
            self.inventory = []
            self.model = self._model()
            self.gamma = 0.95
            self.epsilon = 1.0
            self.epsilon_min = 0.01
            self.epsilon_decay = 0.995
            self.loss_l = []
            self.val_loss_l = []

        def _model(self):
            model = Sequential()
            model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
            model.add(Dense(units=32, activation="relu"))
            model.add(Dense(units=8, activation="relu"))
            model.add(Dense(self.action_size, activation="linear"))
            model.compile(loss="mse", optimizer=Adam(learning_rate=0.001))
            return model

        def act(self, state):
            if random.random()<= self.epsilon:
                return random.randrange(self.action_size)
            options = self.model.predict(state, verbose=0)
            return np.argmax(options[0])

        def opt_model(self):

            state_l = []
            y_f_l = []

            for state, action, reward, next_state in self.memory:

                state_l.append(state)
                y = reward + self.gamma * np.amax(self.model.predict(np.array([next_state]),verbose=0)[0])
                y_f = self.model.predict(np.array([state]), verbose=0)
                y_f[0][action] = y
                y_f_l.append(y_f)

            train_history = self.model.fit(np.array(state_l), np.array(y_f_l), epochs=1, verbose=0)
            self.loss_l.append(train_history.history['loss'])
            #self.val_loss_l.append(train_history.history['val_loss'])

            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

In [6]:
def getState(data, t, n):
    window = data[5+t-n+1:5+t + 1]
    return window

In [27]:
window_size = 5  # state space size, consist with previous window_size days
agent = Agent()
t_p = []
data_hist = np.empty(1)
data_hist[-1] = yf.Ticker('EURUSD=X').info['ask'] 

for i in range(2):

    #state = getState(data_hist, len(data_hist) -1 - window_size, window_size)
    state = []
    for k in range(window_size):
        current = yf.Ticker('EURUSD=X').info['ask']
        state.append(current)
        data_hist = np.append(data_hist, current)
        time.sleep(2)

    total_profit = 0
    agent.inventory = []
    for t in range(5):
        action = agent.act(state)
        reward = 0
        current_price = yf.Ticker('EURUSD=X').info['ask']
        # sit
        if action == 0:
            print("Sit")
        # buy
        elif action == 1:
            if len(agent.inventory) == 0:
                agent.inventory.append(current_price)
            else:
                agent.inventory[0] += current_price
            print("Buy: " + str(current_price))
        # sell
        elif action == 2 and len(agent.inventory) > 0:
            bought_price = window_size_price = agent.inventory.pop(0)
            reward = max(current_price - bought_price, 0)
            total_profit += current_price - bought_price
            t_p.append(total_profit)
            print("Sell: " + str(current_price) + " | Profit: " + str(float(current_price - bought_price)))

        if action == 2 and len(agent.inventory) >= 0:
            print("Sell on empty inventory")

        data_hist = np.append(data_hist, current_price)
        #next_state = getState(data_hist, len(data_hist) -1 - window_size, window_size)
        next_state = data_hist[-5:]
        #done = False
        agent.memory.append((state, action, reward, next_state))
        state = next_state

        time.sleep(2)

    print("-----------------Policy optimization -" + str(i) + "- ----------------")
    agent.opt_model()

Sell on empty inventory
Sell on empty inventory
Sit
Buy: 1.1007154
Sell: 1.1007154 | Profit: 0.0
Sell on empty inventory
-----------------Policy optimization -0- ----------------
Sit
Buy: 1.1007154
Sit
Buy: 1.1007154
Sit
-----------------Policy optimization -1- ----------------
